<a href="https://colab.research.google.com/github/rudoletz/govmomi-examples/blob/master/O'Reilly_Non_linear_ML_Class_Random_Forest_Classification_and_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Market Data

In [ ]:
#Import Python Libraries
import numpy as np
import pandas as pd
from datetime import datetime

import pandas_datareader.data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
#Import data
start = datetime(2020, 1, 1)
end = datetime(2020, 8, 4)

stock = pdr.get_data_yahoo('AAPL', start, end) #Apple Inc. stock
market = pdr.get_data_yahoo('SPY', start, end) #S&P 500 index
vix = pdr.get_data_yahoo('^VIX', start, end)   #Volatility index
dxy = pdr.get_data_yahoo('UUP', start, end)    #Dollar index
junk = pdr.get_data_yahoo('JNK', start, end)   #Junk bond index

In [ ]:
#Create target dataframe
target = pd.DataFrame()
#Use adjusted closing prices instead of closing prices to adjust for corporate actions such as dividends, splits and mergers
target['return'] = (stock['Open']-stock['Adj Close'].shift(1))/stock['Adj Close'].shift(1) #Returns based on buying on the close the day before and selling on the open the day after
target = target.dropna() #get rid of the NaNs
target['direction'] = np.where(target['return'] > 0, 1, -1) #Overnight direction of the stock
target.head()

,return,direction
Date,,
2020-01-03,-0.005622,-1
2020-01-06,-0.007214,-1
2020-01-07,0.005221,1
2020-01-08,0.000944,1
2020-01-09,0.018513,1


In [ ]:
#Create features dataframe based on closing prices
features = pd.DataFrame()
features['market'] = market['Adj Close'].pct_change(1)*100
#VIX is volatility index and is measured in annualizzed percentage terms
features['vix'] = vix['Adj Close'].diff()
features['dxy'] = dxy['Adj Close'].pct_change(1)*100
features['junk'] = junk['Adj Close'].pct_change(1)*100
features = features.dropna()
features.tail()

,market,vix,dxy,junk
Date,,,,
2020-07-28,-0.634239,0.700001,0.158044,-0.313307
2020-07-29,1.229873,-1.340001,-0.473373,0.599996
2020-07-30,-0.356793,0.660000,-0.396358,0.302950
2020-07-31,0.790220,-0.300001,0.596904,0.141572
2020-08-03,0.695216,-0.179998,0.000000,0.014206


In [ ]:
lastknown = features[-1:] #Values of features from the last trading session
features = features[:-1] #Subtracts last row from the features matrix so that it aligns with labels vector
features.tail()

,market,vix,dxy,junk
Date,,,,
2020-07-27,0.729243,-1.100000,-0.745102,0.257001
2020-07-28,-0.634239,0.700001,0.158044,-0.313307
2020-07-29,1.229873,-1.340001,-0.473373,0.599996
2020-07-30,-0.356793,0.660000,-0.396358,0.302950
2020-07-31,0.790220,-0.300001,0.596904,0.141572


#Random Forest classification for overnight direction

In [ ]:
#Get rid of return column for classifiers
targetclass = target.drop(axis=1, columns='return')
targetclass = targetclass[1:] #Removes the first row of labels since we are correlating today's features with tomorrow's opening values
targetclass.head()                                                                        

,direction
Date,
2020-01-06,-1
2020-01-07,1
2020-01-08,1
2020-01-09,1
2020-01-10,1


In [ ]:
#Train and test Random Forest classifier using Gini impurity performance metric
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

classifier = RandomForestClassifier(random_state=1, oob_score=True) #Enables using out-of-bag sample set for validation
targetclass = np.ravel(targetclass) #Need to covert column vector into a 1-d Numpy array 
features_train, features_test, targetclass_train, targetclass_test = train_test_split(features, targetclass, test_size = 0.25, random_state=0)
classifier.fit(features_train, targetclass_train)
print("Training score:", classifier.score(features_train, targetclass_train))
print("Out-of-bag score:", classifier.oob_score_)
print("Testing score:", classifier.score(features_test, targetclass_test))

Training score: 1.0
Out-of-bag score: 0.47706422018348627
Testing score: 0.6216216216216216


In [ ]:
#Train and test classifier using entropy performance metric with regularizing hyperparameters
classifier_entropy = RandomForestClassifier(criterion='entropy', random_state=1, oob_score=True,n_estimators=100, max_depth=5, min_samples_leaf=20, min_samples_split=30) #Maximum depth of tree is used to prevent overfitting of test data 
features_train, features_test, targetclass_train, targetclass_test = train_test_split(features, targetclass, test_size = 0.25, random_state=0)
classifier_entropy.fit(features_train, targetclass_train)
print("Training score:", classifier_entropy.score(features_train, targetclass_train))
print("Out-of-bag score:", classifier_entropy.oob_score_)
print("Testing score:", classifier_entropy.score(features_test, targetclass_test))

Training score: 0.5963302752293578
Out-of-bag score: 0.5963302752293578
Testing score: 0.6216216216216216


In [ ]:
from sklearn.metrics import confusion_matrix
targetclass_predict = classifier_entropy.predict(features_test)
confusion = confusion_matrix(targetclass_test, targetclass_predict)
confusion

array([[ 0, 14],
       [ 0, 23]])

In [ ]:
print("Tomorrow's change:", classifier_entropy.predict(lastknown))
print("Probability of change", classifier_entropy.predict_proba(lastknown))
lastknown

Tomorrow's change: [1]
Probability of change [[0.35932921 0.64067079]]


,market,vix,dxy,junk
Date,,,,
2020-08-03,0.695216,-0.179998,0.0,0.014206


In [ ]:
#Inferring the importance of each feature
print(features.columns)
print(classifier.feature_importances_)
print(classifier_entropy.feature_importances_)

Index(['market', 'vix', 'dxy', 'junk'], dtype='object')
[0.20283906 0.24788042 0.30785675 0.24142377]
[0.21991413 0.29285605 0.31442419 0.17280562]


#Random Forest regression for overnight value changes

In [ ]:
#Get rid of direction column for regressors
targetvalue = target.drop(axis=1, columns='direction')
targetvalue = targetvalue[1:] #Removes the first row of labels since we are correlating today's features with tomorrow's opening values
targetvalue.head()                                                                        


,return
Date,
2020-01-06,-0.007214
2020-01-07,0.005221
2020-01-08,0.000944
2020-01-09,0.018513
2020-01-10,0.008235


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

regressor = RandomForestRegressor(random_state=1, oob_score=True) #default performance metric is mean square error
targetvalue = np.ravel(targetvalue) #Need to covert column vector into a 1-d array
features_train, features_test, targetvalue_train, targetvalue_test = train_test_split(features, targetvalue, test_size = 0.25, random_state=0)
regressor.fit(features_train, targetvalue_train)
print("Training score:", regressor.score(features_train, targetvalue_train))
print("Out-of-bag score:", regressor.oob_score_)
print("Testing score:", regressor.score(features_test, targetvalue_test))
print("Tomorrow's value change:", regressor.predict(lastknown))

Training score: 0.8455460297724262
Out-of-bag score: -0.32901912043396275
Testing score: -0.041938586284235635
Tomorrow's value change: [0.00259391]


In [ ]:
regressor_mae = RandomForestRegressor(criterion="mae", random_state=1, oob_score=True, max_depth=10, n_estimators=500)
features_train, features_test, targetvalue_train, targetvalue_test = train_test_split(features, targetvalue, test_size = 0.25, random_state=0)
regressor_mae.fit(features_train, targetvalue_train)
print("Training score:", regressor_mae.score(features_train, targetvalue_train))
print("Out-of-bag score:", regressor_mae.oob_score_)
print("Testing score:", regressor_mae.score(features_test, targetvalue_test))
print("Tomorrow's value change:", regressor_mae.predict(lastknown))

Training score: 0.7986063918839404
Out-of-bag score: -0.21310314083538295
Testing score: 0.023638115387403458
Tomorrow's value change: [0.0033212]


In [ ]:
#Inferring the importance of each feature
print(features.columns)
print(regressor.feature_importances_)
print(regressor_mae.feature_importances_)

Index(['market', 'vix', 'dxy', 'junk'], dtype='object')
[0.24583685 0.36303334 0.27569427 0.11543553]
[0.27640709 0.2583581  0.29494468 0.17029013]
